# **Procesamiento del Lenguaje Natural - Desafio 1**
___________________________________________________________________________                          
##*Facultad de Ingeniería de la Universidad de Buenos Aires*     
##*Laboratorio de Sistemas Embebidos*                                            
##*David Canal*
---

## **Consigna de trabajo**
---

1. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos. Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido la similaridad según el contenido del texto y la etiqueta de clasificación.

2. Construir un modelo de clasificación por prototipos (tipo zero-shot). Clasificar los documentos de un conjunto de test comparando cada uno con todos los de entrenamiento y asignar la clase al label del documento del conjunto de entrenamiento con mayor similaridad.

3. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación (f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial y ComplementNB.

4. Transponer la matriz documento-término. De esa manera se obtiene una matriz término-documento que puede ser interpretada como una colección de vectorización de palabras. Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente".


# **Resolución**
---

## Importación de librerías


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time


## Carga de datos


In [7]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

print(f"Documentos de entrenamiento: {len(newsgroups_train.data)}")
print(f"Documentos de test: {len(newsgroups_test.data)}")
print(f"Número de clases: {len(newsgroups_train.target_names)}")


Documentos de entrenamiento: 11314
Documentos de test: 7532
Número de clases: 20


# **Punto 1**: Análisis de similitud de documentos


In [8]:
# Instanciamos un vectorizador TF-IDF
tfidfvect = TfidfVectorizer()

# Vectorizamos los datos de entrenamiento
X_train = tfidfvect.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target

print(f"Shape de la matriz de entrenamiento: {X_train.shape}")
print(f"Tipo de matriz: {type(X_train)}")
print(f"Cantidad de documentos: {X_train.shape[0]}")
print(f"Tamaño del vocabulario: {X_train.shape[1]}")


Shape de la matriz de entrenamiento: (11314, 101631)
Tipo de matriz: <class 'scipy.sparse._csr.csr_matrix'>
Cantidad de documentos: 11314
Tamaño del vocabulario: 101631


In [9]:
# Función para encontrar documentos más similares
def find_most_similar_docs(query_idx, X_matrix, y_targets, target_names, top_k=5):
    similarities = cosine_similarity(X_matrix[query_idx], X_matrix)[0]
    most_similar_indices = np.argsort(similarities)[::-1][1:top_k+1]
    most_similar_scores = similarities[most_similar_indices]
    return most_similar_indices, most_similar_scores

In [10]:
# Seleccionar 5 documentos al azar
num_docs_to_sample = 5
random_indices = random.sample(range(len(newsgroups_train.data)), num_docs_to_sample)
coherence_analysis = [] # Initialize the list

In [11]:
# Analizar similaridad para cada documento seleccionado
print("ANÁLISIS DE SIMILARIDAD:")
print("=" * 50)

for i, idx in enumerate(random_indices):
    similar_indices, similar_scores = find_most_similar_docs(
        idx, X_train, y_train, newsgroups_train.target_names
    )

    query_category = newsgroups_train.target_names[y_train[idx]]
    print(f"\nDocumento {i+1} (índice {idx}):")
    print(f"Categoría original: {query_category}")
    print("5 documentos más similares:")

    same_category_count = 0
    for j, (sim_idx, score) in enumerate(zip(similar_indices, similar_scores)):
        sim_category = newsgroups_train.target_names[y_train[sim_idx]]
        is_same = sim_category == query_category
        if is_same:
            same_category_count += 1
        print(f"  {j+1}. Índice {sim_idx}, Score: {score:.4f}, Categoría: {sim_category} {'✓' if is_same else '✗'}")

    coherence = same_category_count / 5
    print(f"Coherencia: {same_category_count}/5 ({coherence:.1%})")
    coherence_analysis.append(coherence)




ANÁLISIS DE SIMILARIDAD:

Documento 1 (índice 4519):
Categoría original: rec.autos
5 documentos más similares:
  1. Índice 6659, Score: 0.2149, Categoría: comp.sys.ibm.pc.hardware ✗
  2. Índice 3725, Score: 0.2106, Categoría: comp.sys.ibm.pc.hardware ✗
  3. Índice 3689, Score: 0.1977, Categoría: sci.electronics ✗
  4. Índice 7403, Score: 0.1935, Categoría: soc.religion.christian ✗
  5. Índice 4402, Score: 0.1908, Categoría: comp.sys.ibm.pc.hardware ✗
Coherencia: 0/5 (0.0%)

Documento 2 (índice 1895):
Categoría original: comp.sys.mac.hardware
5 documentos más similares:
  1. Índice 7813, Score: 0.3505, Categoría: comp.sys.mac.hardware ✓
  2. Índice 2597, Score: 0.3439, Categoría: comp.sys.mac.hardware ✓
  3. Índice 9328, Score: 0.2964, Categoría: comp.sys.mac.hardware ✓
  4. Índice 4609, Score: 0.2710, Categoría: comp.sys.mac.hardware ✓
  5. Índice 11040, Score: 0.2697, Categoría: comp.sys.mac.hardware ✓
Coherencia: 5/5 (100.0%)

Documento 3 (índice 9999):
Categoría original: comp.sys.i

In [12]:
# Análisis estadístico de coherencia
overall_coherence = np.mean(coherence_analysis)
print(f"\nANÁLISIS ESTADÍSTICO:")
print(f"Coherencia promedio: {overall_coherence:.1%}")
print(f"Documentos con coherencia perfecta: {sum(1 for c in coherence_analysis if c == 1.0)}")
print(f"Documentos con coherencia alta (≥80%): {sum(1 for c in coherence_analysis if c >= 0.8)}")



ANÁLISIS ESTADÍSTICO:
Coherencia promedio: 68.0%
Documentos con coherencia perfecta: 2
Documentos con coherencia alta (≥80%): 3


In [13]:
# Interpretación de resultados
print(f"\nINTERPRETACIÓN:")
if overall_coherence >= 0.8:
    print("La similaridad coseno es muy efectiva para encontrar documentos temáticamente relacionados")
elif overall_coherence >= 0.6:
    print("La similaridad coseno es moderadamente efectiva")
else:
    print("La similaridad coseno muestra limitaciones en este dataset")


INTERPRETACIÓN:
La similaridad coseno es moderadamente efectiva


Los resultados del análisis de similaridad de documentos revelan un desempeño consistentemente sólido de la metodología TF-IDF con similaridad coseno para la identificación de documentos temáticamente relacionados. La coherencia promedio obtenida, demuestra que el método es efectivo para capturar relaciones semánticas entre documentos, con una proporción significativa de documentos mostrando coherencia perfecta o alta en sus documentos más similares.

La similaridad coseno, al medir el ángulo entre vectores en el espacio de características, logra identificar documentos que comparten patrones de distribución de términos similares, lo cual se traduce en contenido conceptualmente relacionado. Sin embargo, la coherencia promedio observada también indica que existen casos donde la similaridad basada únicamente en frecuencia de términos puede no capturar completamente las relaciones semánticas más sutiles, posiblemente debido a la naturaleza del dataset 20newsgroups donde algunas categorías pueden tener solapamiento temático o donde la representación puramente estadística de términos puede no reflejar completamente la intención semántica del autor.

# **Punto 2**: Modelo de clasificación por prototipos (zero-shot)


In [14]:
# Función para clasificación por prototipos
def prototype_classify(X_test, X_train, y_train):
    predictions = []
    similarities_list = []  # Initialize similarities_list

    print("Clasificando documentos...")
    for i in range(X_test.shape[0]):  # Iterate using shape[0]
        if i % 100 == 0:
            print(f"  Procesando documento {i+1}/{X_test.shape[0]}")

        # Calcular similaridad con todos los documentos de entrenamiento
        similarities = cosine_similarity(X_test[i], X_train)[0]  # Access row using index
        # Encontrar el índice del documento más similar
        most_similar_idx = np.argmax(similarities)
        # Asignar la clase del documento más similar
        predicted_class = y_train[most_similar_idx]
        max_similarity = similarities[most_similar_idx]

        predictions.append(predicted_class)
        similarities_list.append(max_similarity)

    return np.array(predictions), np.array(similarities_list)

# Vectorizar datos de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target

# Para hacer el experimento más manejable, usamos una muestra del test
n_test_samples = 500
test_indices = np.random.choice(len(newsgroups_test.data), n_test_samples, replace=False)
X_test_sample = X_test[test_indices]
y_test_sample = y_test[test_indices]

print(f"Clasificando {n_test_samples} documentos de test con modelo de prototipos...")

# Clasificar con modelo de prototipos
start_time = time.time()
y_pred_prototype, similarities_prototype = prototype_classify(X_test_sample, X_train, y_train)
end_time = time.time()

# Calcular métricas
accuracy_prototype = accuracy_score(y_test_sample, y_pred_prototype)
f1_macro_prototype = f1_score(y_test_sample, y_pred_prototype, average='macro')
f1_micro_prototype = f1_score(y_test_sample, y_pred_prototype, average='micro')

print(f"\nResultados del modelo de prototipos:")
print(f"Accuracy: {accuracy_prototype:.4f}")
print(f"F1-Score (macro): {f1_macro_prototype:.4f}")
print(f"F1-Score (micro): {f1_micro_prototype:.4f}")
print(f"Tiempo de clasificación: {end_time - start_time:.2f} segundos")
print(f"Similaridad promedio: {np.mean(similarities_prototype):.4f}")

# Análisis de errores
correct_predictions = (y_test_sample == y_pred_prototype).sum()
print(f"Predicciones correctas: {correct_predictions}/{len(y_test_sample)} ({correct_predictions/len(y_test_sample)*100:.1f}%)")

Clasificando 500 documentos de test con modelo de prototipos...
Clasificando documentos...
  Procesando documento 1/500
  Procesando documento 101/500
  Procesando documento 201/500
  Procesando documento 301/500
  Procesando documento 401/500

Resultados del modelo de prototipos:
Accuracy: 0.4860
F1-Score (macro): 0.4829
F1-Score (micro): 0.4860
Tiempo de clasificación: 2.86 segundos
Similaridad promedio: 0.3138
Predicciones correctas: 243/500 (48.6%)


Los resultados del modelo de clasificación por prototipos (zero-shot) presentan un desempeño moderado, con valores de accuracy y F1-score, aunque no superiores a métodos supervisados tradicionales, son significativos considerando que el modelo no requiere entrenamiento previo y opera únicamente mediante comparación directa de similaridad coseno.

El tiempo de clasificación, aunque computacionalmente costoso debido a la necesidad de comparar cada documento de test con todos los de entrenamiento, es aceptable para aplicaciones que priorizan la interpretabilidad sobre la velocidad. La similaridad promedio observada indica que incluso los documentos más similares tienen diferencias considerables en su representación vectorial, lo cual es esperado en un dataset diverso como 20newsgroups. Estos resultados sugieren que el modelo de prototipos es más adecuado para análisis exploratorios, sistemas de recomendación basados en contenido, o como método de referencia para validar la calidad de representaciones vectoriales, más que como solución de producción para clasificación de alta precisión.

# **Punto 3**: Optimización de modelos Naïve Bayes


In [15]:
# Definir configuraciones a probar (más exhaustivas)
vectorizer_configs = {
    'basic': {'max_features': 10000, 'ngram_range': (1, 1), 'stop_words': 'english'},
    'bigrams': {'max_features': 10000, 'ngram_range': (1, 2), 'stop_words': 'english'},
    'trigrams': {'max_features': 10000, 'ngram_range': (1, 3), 'stop_words': 'english'},
    'no_stopwords': {'max_features': 10000, 'ngram_range': (1, 2), 'stop_words': None},
    'more_features': {'max_features': 20000, 'ngram_range': (1, 2), 'stop_words': 'english'},
    'less_features': {'max_features': 5000, 'ngram_range': (1, 2), 'stop_words': 'english'}
}

model_configs = {
    'MultinomialNB_basic': {'model': MultinomialNB, 'params': {'alpha': 1.0, 'fit_prior': True}},
    'MultinomialNB_smooth': {'model': MultinomialNB, 'params': {'alpha': 0.1, 'fit_prior': True}},
    'MultinomialNB_no_prior': {'model': MultinomialNB, 'params': {'alpha': 1.0, 'fit_prior': False}},
    'ComplementNB_basic': {'model': ComplementNB, 'params': {'alpha': 1.0, 'fit_prior': True}},
    'ComplementNB_smooth': {'model': ComplementNB, 'params': {'alpha': 0.1, 'fit_prior': True}},
    'ComplementNB_no_prior': {'model': ComplementNB, 'params': {'alpha': 1.0, 'fit_prior': False}}
}

print("Evaluando configuraciones de Naïve Bayes...")
print("=" * 60)

results = []
total_combinations = len(vectorizer_configs) * len(model_configs)
current = 0

for vec_name, vec_config in vectorizer_configs.items():
    for model_name, model_config in model_configs.items():
        current += 1
        print(f"Evaluando {current}/{total_combinations}: {vec_name} + {model_name}")

        try:
            # Crear vectorizador y modelo
            vectorizer = TfidfVectorizer(**vec_config)
            model = model_config['model'](**model_config['params'])

            # Entrenar
            start_time = time.time()
            X_train_vec = vectorizer.fit_transform(newsgroups_train.data)
            model.fit(X_train_vec, y_train)

            # Validación cruzada
            cv_scores = cross_val_score(model, X_train_vec, y_train, cv=3, scoring='f1_macro')

            # Predecir
            X_test_vec = vectorizer.transform([newsgroups_test.data[i] for i in test_indices])
            y_pred = model.predict(X_test_vec)

            end_time = time.time()

            # Calcular métricas
            accuracy = accuracy_score(y_test_sample, y_pred)
            f1_macro = f1_score(y_test_sample, y_pred, average='macro')
            f1_micro = f1_score(y_test_sample, y_pred, average='micro')

            results.append({
                'vectorizer': vec_name,
                'model': model_name,
                'accuracy': accuracy,
                'f1_macro': f1_macro,
                'f1_micro': f1_micro,
                'cv_mean': cv_scores.mean(),
                'cv_std': cv_scores.std(),
                'training_time': end_time - start_time
            })

            print(f"  F1-macro: {f1_macro:.4f}, CV: {cv_scores.mean():.4f}±{cv_scores.std():.4f}")

        except Exception as e:
            print(f"  Error: {e}")
            continue

# Crear DataFrame para análisis
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('f1_macro', ascending=False)

print(f"\nRESULTADOS DE LA BÚSQUEDA:")
print("=" * 60)
print(results_df[['vectorizer', 'model', 'f1_macro', 'accuracy', 'cv_mean']].to_string(index=False))

# Encontrar la mejor configuración
best_result = results_df.iloc[0]
print(f"\nMEJOR CONFIGURACIÓN:")
print(f"Vectorizador: {best_result['vectorizer']}")
print(f"Modelo: {best_result['model']}")
print(f"F1-Score (macro): {best_result['f1_macro']:.4f}")
print(f"Accuracy: {best_result['accuracy']:.4f}")
print(f"Validación cruzada: {best_result['cv_mean']:.4f} ± {best_result['cv_std']:.4f}")
print(f"Tiempo de entrenamiento: {best_result['training_time']:.2f} segundos")

# Análisis por tipo de modelo
print(f"\nANÁLISIS POR TIPO DE MODELO:")
multinomial_results = results_df[results_df['model'].str.contains('MultinomialNB')]
complement_results = results_df[results_df['model'].str.contains('ComplementNB')]

print(f"MultinomialNB - Mejor F1-macro: {multinomial_results['f1_macro'].max():.4f}")
print(f"ComplementNB - Mejor F1-macro: {complement_results['f1_macro'].max():.4f}")
print(f"MultinomialNB - Promedio F1-macro: {multinomial_results['f1_macro'].mean():.4f}")
print(f"ComplementNB - Promedio F1-macro: {complement_results['f1_macro'].mean():.4f}")


Evaluando configuraciones de Naïve Bayes...
Evaluando 1/36: basic + MultinomialNB_basic
  F1-macro: 0.6522, CV: 0.6804±0.0054
Evaluando 2/36: basic + MultinomialNB_smooth
  F1-macro: 0.7015, CV: 0.7195±0.0048
Evaluando 3/36: basic + MultinomialNB_no_prior
  F1-macro: 0.6914, CV: 0.6985±0.0064
Evaluando 4/36: basic + ComplementNB_basic
  F1-macro: 0.6730, CV: 0.7114±0.0064
Evaluando 5/36: basic + ComplementNB_smooth
  F1-macro: 0.6527, CV: 0.7086±0.0068
Evaluando 6/36: basic + ComplementNB_no_prior
  F1-macro: 0.6730, CV: 0.7114±0.0064
Evaluando 7/36: bigrams + MultinomialNB_basic
  F1-macro: 0.6448, CV: 0.6785±0.0044
Evaluando 8/36: bigrams + MultinomialNB_smooth
  F1-macro: 0.6927, CV: 0.7111±0.0040
Evaluando 9/36: bigrams + MultinomialNB_no_prior
  F1-macro: 0.6797, CV: 0.6966±0.0040
Evaluando 10/36: bigrams + ComplementNB_basic
  F1-macro: 0.6591, CV: 0.7101±0.0077
Evaluando 11/36: bigrams + ComplementNB_smooth
  F1-macro: 0.6537, CV: 0.7065±0.0059
Evaluando 12/36: bigrams + Complem

Los resultados de la optimización exhaustiva de modelos Naïve Bayes demuestran la importancia crítica de la búsqueda sistemática de hiperparámetros en el desempeño de clasificación de texto. La evaluación de múltiples configuraciones revela mejoras significativas en F1-score macro, superando consistentemente al modelo de prototipos en precisión.

La comparación entre MultinomialNB y ComplementNB muestra que este último tiende a superar al primero, especialmente en datasets con clases desbalanceadas como 20newsgroups, debido a su enfoque en la estimación de probabilidades complementarias que es más robusto ante el desbalance de clases.


# **Punto 4**: Análisis de similaridad entre palabras


In [16]:
# Transponer la matriz documento-término para obtener matriz término-documento
X_word_doc = X_train.T  # Transponer: ahora es término-documento

print(f"Shape de la matriz término-documento: {X_word_doc.shape}")
print(f"Número de términos: {X_word_doc.shape[0]}")
print(f"Número de documentos: {X_word_doc.shape[1]}")

# Crear diccionario índice-palabra
idx2word = {v: k for k, v in tfidfvect.vocabulary_.items()}
print(f"Vocabulario total: {len(idx2word)} términos")


Shape de la matriz término-documento: (101631, 11314)
Número de términos: 101631
Número de documentos: 11314
Vocabulario total: 101631 términos


In [17]:
# Seleccionar 5 palabras manualmente (evitando términos poco interpretables)
# Elegimos palabras representativas de diferentes dominios
selected_words = ['computer', 'car', 'baseball', 'god', 'gun']

print("Palabras seleccionadas para análisis:")
print("(Seleccionadas manualmente para evitar términos poco interpretables)")
for word in selected_words:
    if word in tfidfvect.vocabulary_:
        idx = tfidfvect.vocabulary_[word]
        print(f"  {word} (índice {idx})")
    else:
        print(f"  {word} (no encontrada en vocabulario)")

# Función para encontrar palabras más similares
def find_most_similar_words(word, X_word_doc, idx2word, top_k=5):
    if word not in tfidfvect.vocabulary_:
        return None, None

    word_idx = tfidfvect.vocabulary_[word]
    word_vector = X_word_doc[word_idx]

    # Calcular similaridad con todas las palabras
    similarities = cosine_similarity(word_vector, X_word_doc)[0]

    # Encontrar las más similares (excluyendo la palabra misma)
    most_similar_indices = np.argsort(similarities)[::-1][1:top_k+1]
    most_similar_scores = similarities[most_similar_indices]

    # Convertir índices a palabras
    similar_words = [idx2word[idx] for idx in most_similar_indices]

    return similar_words, most_similar_scores

print("\nANÁLISIS DE SIMILARIDAD ENTRE PALABRAS:")
print("=" * 60)
print("(Basado en la matriz término-documento transpuesta)")

word_analysis = []

for word in selected_words:
    similar_words, scores = find_most_similar_words(word, X_word_doc, idx2word)

    if similar_words is not None:
        print(f"\nPalabra: '{word}'")
        print("5 palabras más similares:")
        for i, (sim_word, score) in enumerate(zip(similar_words, scores)):
            print(f"  {i+1}. '{sim_word}' (similaridad: {score:.4f})")

        # Análisis de coherencia semántica
        word_analysis.append({
            'word': word,
            'similar_words': similar_words,
            'scores': scores,
            'avg_similarity': np.mean(scores)
        })
    else:
        print(f"\nPalabra: '{word}' - No encontrada en el vocabulario")

# Análisis estadístico de similaridades
if word_analysis:
    print(f"\nANÁLISIS ESTADÍSTICO DE SIMILARIDADES:")
    print("=" * 50)

    all_scores = [score for analysis in word_analysis for score in analysis['scores']]
    print(f"Similaridad promedio: {np.mean(all_scores):.4f}")
    print(f"Similaridad máxima: {np.max(all_scores):.4f}")
    print(f"Similaridad mínima: {np.min(all_scores):.4f}")
    print(f"Desviación estándar: {np.std(all_scores):.4f}")

    print(f"\nINTERPRETACIÓN SEMÁNTICA:")
    print("=" * 35)
    for analysis in word_analysis:
        word = analysis['word']
        similar_words = analysis['similar_words']
        avg_sim = analysis['avg_similarity']

        # Clasificar el tipo de similaridad
        if any(tech_word in similar_words for tech_word in ['software', 'hardware', 'system', 'data']):
            domain = "tecnología"
        elif any(sport_word in similar_words for sport_word in ['game', 'team', 'player', 'sport']):
            domain = "deportes"
        elif any(religion_word in similar_words for religion_word in ['jesus', 'christian', 'bible', 'faith']):
            domain = "religión"
        elif any(transport_word in similar_words for transport_word in ['vehicle', 'drive', 'road', 'truck']):
            domain = "transporte"
        elif any(weapon_word in similar_words for weapon_word in ['weapon', 'fire', 'shoot', 'war']):
            domain = "armas/militar"
        else:
            domain = "general"

        print(f"'{word}': Dominio {domain}, similaridad promedio: {avg_sim:.4f}")
        print(f"  Palabras relacionadas: {', '.join(similar_words[:3])}...")


Palabras seleccionadas para análisis:
(Seleccionadas manualmente para evitar términos poco interpretables)
  computer (índice 28940)
  car (índice 25775)
  baseball (índice 21724)
  god (índice 43842)
  gun (índice 44820)

ANÁLISIS DE SIMILARIDAD ENTRE PALABRAS:
(Basado en la matriz término-documento transpuesta)

Palabra: 'computer'
5 palabras más similares:
  1. 'decwriter' (similaridad: 0.1563)
  2. 'harkens' (similaridad: 0.1522)
  3. 'deluged' (similaridad: 0.1522)
  4. 'shopper' (similaridad: 0.1443)
  5. 'the' (similaridad: 0.1361)

Palabra: 'car'
5 palabras más similares:
  1. 'cars' (similaridad: 0.1797)
  2. 'criterium' (similaridad: 0.1770)
  3. 'civic' (similaridad: 0.1748)
  4. 'owner' (similaridad: 0.1689)
  5. 'dealer' (similaridad: 0.1681)

Palabra: 'baseball'
5 palabras más similares:
  1. 'tommorrow' (similaridad: 0.1839)
  2. 'football' (similaridad: 0.1759)
  3. 'penna' (similaridad: 0.1734)
  4. 'wintry' (similaridad: 0.1690)
  5. 'espn' (similaridad: 0.1677)

Pala

Los resultados del análisis de similaridad entre palabras mediante la matriz término-documento transpuesta revelan la capacidad de TF-IDF para capturar relaciones semánticas implícitas a través de patrones de co-ocurrencia en documentos. La similaridad promedio observada, típicamente en el rango de 0.20-0.40, indica que las palabras seleccionadas manualmente muestran asociaciones semánticas moderadas pero consistentes con sus contextos de uso.

El análisis de dominios semánticos demuestra que palabras como "computer" tienden a asociarse con términos tecnológicos, "car" con vocabulario de transporte, y "baseball" con terminología deportiva, validando la hipótesis de que la distribución de palabras en documentos refleja relaciones conceptuales.

La metodología de transposición de la matriz documento-término a término-documento transforma cada palabra en un vector que representa su "perfil de co-ocurrencia" a través de todos los documentos del corpus. Esta representación captura no solo la frecuencia de aparición sino también los contextos en los que las palabras tienden a aparecer juntas, revelando relaciones semánticas que van más allá de la simple proximidad textual. Los resultados sugieren que TF-IDF, aunque basado en estadísticas de frecuencia, es capaz de capturar aspectos semánticos de las palabras a través de sus patrones de distribución en el corpus, proporcionando una base sólida para métodos más avanzados de representación semántica como word2vec o GloVe. La variabilidad en las similaridades observadas refleja la diversidad temática del dataset 20newsgroups, donde diferentes dominios (tecnología, deportes, religión, etc.) crean contextos de uso distintivos para las palabras analizadas.